In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import os
import pickle
import pandas as pd
import random
from sklearn.utils import shuffle
from matplotlib import pyplot as plt
import datetime as dt
import time

pd.set_option('mode.chained_assignment', None)
max_vel=500;
x_city_min = 3750901.5068
x_city_max = 3770901.5068
y_city_min =-19268905.6133
y_city_max =-19208905.6133

column_names = ['','hash','trajectory_id','time_entry','time_exit','vmax','vmin','vmean','x_entry','y_entry','x_exit','y_exit']
col_types = {'x_entry': np.float64}

df = pd.read_csv("data_train.csv", names=column_names,  skiprows=1, dtype = col_types)

print(df.head())
df.describe()
#df.dtypes

                                    hash  \
0  0  0000a8602cf2def930488dee7cdad104_1   
1  1  0000a8602cf2def930488dee7cdad104_1   
2  2  0000a8602cf2def930488dee7cdad104_1   
3  3  0000a8602cf2def930488dee7cdad104_1   
4  4  0000a8602cf2def930488dee7cdad104_1   

                               trajectory_id time_entry time_exit  vmax  vmin  \
0  traj_0000a8602cf2def930488dee7cdad104_1_0   07:04:31  07:08:32   NaN   NaN   
1  traj_0000a8602cf2def930488dee7cdad104_1_1   07:20:34  07:25:42   NaN   NaN   
2  traj_0000a8602cf2def930488dee7cdad104_1_2   07:53:32  08:03:25   NaN   NaN   
3  traj_0000a8602cf2def930488dee7cdad104_1_3   08:17:50  08:37:23   NaN   NaN   
4  traj_0000a8602cf2def930488dee7cdad104_1_4   14:38:09  14:38:09   NaN   NaN   

   vmean       x_entry       y_entry        x_exit        y_exit  
0    NaN  3.751014e+06 -1.909398e+07  3.750326e+06 -1.913634e+07  
1    NaN  3.743937e+06 -1.932247e+07  3.744975e+06 -1.931966e+07  
2    NaN  3.744868e+06 -1.929356e+07  3.744816e

,,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit
count,814262.000000,256769.000000,256769.000000,270778.000000,8.142620e+05,8.142620e+05,8.142620e+05,8.142620e+05
mean,407130.500000,2.680249,2.279469,2.454799,3.760411e+06,-1.922153e+07,3.760426e+06,-1.922181e+07
std,235057.336783,7.591053,7.118452,7.160578,8.974393e+03,7.632234e+04,8.904451e+03,7.515560e+04
min,0.000000,-1.000000,-1.000000,-1.000000,3.741027e+06,-1.938291e+07,3.740998e+06,-1.937688e+07
25%,203565.250000,0.000000,-1.000000,0.000000,3.755174e+06,-1.927454e+07,3.755359e+06,-1.927308e+07
50%,407130.500000,0.000000,0.000000,0.000000,3.760071e+06,-1.923020e+07,3.760076e+06,-1.923031e+07
75%,610695.750000,0.500000,0.220000,0.440000,3.767509e+06,-1.916999e+07,3.767455e+06,-1.917193e+07
max,814261.000000,162.000000,162.000000,162.000000,3.777099e+06,-1.904266e+07,3.777055e+06,-1.904666e+07


In [2]:
#remove velocity columns
df2 = df.drop(columns = ["","vmin", "vmax", "vmean"])
#df2 = df2[1:]

print(df2.head(5))
df2.describe()

                                 hash  \
0  0000a8602cf2def930488dee7cdad104_1   
1  0000a8602cf2def930488dee7cdad104_1   
2  0000a8602cf2def930488dee7cdad104_1   
3  0000a8602cf2def930488dee7cdad104_1   
4  0000a8602cf2def930488dee7cdad104_1   

                               trajectory_id time_entry time_exit  \
0  traj_0000a8602cf2def930488dee7cdad104_1_0   07:04:31  07:08:32   
1  traj_0000a8602cf2def930488dee7cdad104_1_1   07:20:34  07:25:42   
2  traj_0000a8602cf2def930488dee7cdad104_1_2   07:53:32  08:03:25   
3  traj_0000a8602cf2def930488dee7cdad104_1_3   08:17:50  08:37:23   
4  traj_0000a8602cf2def930488dee7cdad104_1_4   14:38:09  14:38:09   

        x_entry       y_entry        x_exit        y_exit  
0  3.751014e+06 -1.909398e+07  3.750326e+06 -1.913634e+07  
1  3.743937e+06 -1.932247e+07  3.744975e+06 -1.931966e+07  
2  3.744868e+06 -1.929356e+07  3.744816e+06 -1.929284e+07  
3  3.744880e+06 -1.929229e+07  3.744809e+06 -1.929049e+07  
4  3.744909e+06 -1.928558e+07  3.74490

,x_entry,y_entry,x_exit,y_exit
count,8.142620e+05,8.142620e+05,8.142620e+05,8.142620e+05
mean,3.760411e+06,-1.922153e+07,3.760426e+06,-1.922181e+07
std,8.974393e+03,7.632234e+04,8.904451e+03,7.515560e+04
min,3.741027e+06,-1.938291e+07,3.740998e+06,-1.937688e+07
25%,3.755174e+06,-1.927454e+07,3.755359e+06,-1.927308e+07
50%,3.760071e+06,-1.923020e+07,3.760076e+06,-1.923031e+07
75%,3.767509e+06,-1.916999e+07,3.767455e+06,-1.917193e+07
max,3.777099e+06,-1.904266e+07,3.777055e+06,-1.904666e+07


In [3]:
df2['time_entry'] = pd.to_timedelta(df2['time_entry']).dt.total_seconds()

df2['time_exit'] = pd.to_timedelta(df2['time_exit']).dt.total_seconds()

#df3['entry_seconds'] = df3['time_entry'].dt.total_seconds()
#df3['exit_seconds'] = df3['exit_dtime'].dt.total_seconds()

#print(df2.head())

                                 hash  \
0  0000a8602cf2def930488dee7cdad104_1   
1  0000a8602cf2def930488dee7cdad104_1   
2  0000a8602cf2def930488dee7cdad104_1   
3  0000a8602cf2def930488dee7cdad104_1   
4  0000a8602cf2def930488dee7cdad104_1   

                               trajectory_id  time_entry  time_exit  \
0  traj_0000a8602cf2def930488dee7cdad104_1_0     25471.0    25712.0   
1  traj_0000a8602cf2def930488dee7cdad104_1_1     26434.0    26742.0   
2  traj_0000a8602cf2def930488dee7cdad104_1_2     28412.0    29005.0   
3  traj_0000a8602cf2def930488dee7cdad104_1_3     29870.0    31043.0   
4  traj_0000a8602cf2def930488dee7cdad104_1_4     52689.0    52689.0   

        x_entry       y_entry        x_exit        y_exit  
0  3.751014e+06 -1.909398e+07  3.750326e+06 -1.913634e+07  
1  3.743937e+06 -1.932247e+07  3.744975e+06 -1.931966e+07  
2  3.744868e+06 -1.929356e+07  3.744816e+06 -1.929284e+07  
3  3.744880e+06 -1.929229e+07  3.744809e+06 -1.929049e+07  
4  3.744909e+06 -1.928558e

In [4]:
df2.describe()

,time_entry,time_exit,x_entry,y_entry,x_exit,y_exit
count,814262.000000,814262.000000,8.142620e+05,8.142620e+05,8.142620e+05,8.142620e+05
mean,39342.660226,39689.670820,3.760411e+06,-1.922153e+07,3.760426e+06,-1.922181e+07
std,13993.416225,14025.396034,8.974393e+03,7.632234e+04,8.904451e+03,7.515560e+04
min,0.000000,0.000000,3.741027e+06,-1.938291e+07,3.740998e+06,-1.937688e+07
25%,30610.000000,31043.000000,3.755174e+06,-1.927454e+07,3.755359e+06,-1.927308e+07
50%,42262.000000,42606.000000,3.760071e+06,-1.923020e+07,3.760076e+06,-1.923031e+07
75%,51175.000000,51530.000000,3.767509e+06,-1.916999e+07,3.767455e+06,-1.917193e+07
max,57600.000000,57600.000000,3.777099e+06,-1.904266e+07,3.777055e+06,-1.904666e+07


In [5]:
Good_Hashs = pickle.load(open("df4.pickle","rb"))
Good_Hashs.describe()

,time_entry,time_exit,x_entry,y_entry,x_exit,y_exit,deltaT,delta_X,vel
count,66762.000000,66762.000000,6.676200e+04,6.676200e+04,6.676200e+04,6.676200e+04,66762.000000,66762.000000,66762.000000
mean,54701.511459,55534.569785,3.760930e+06,-1.922132e+07,3.760896e+06,-1.922034e+07,833.058327,31609.539322,39.667926
std,1229.914855,1013.694149,9.278789e+03,7.596111e+04,9.489489e+03,8.024550e+04,798.387138,47758.353836,56.317935
min,38005.000000,54000.000000,3.741032e+06,-1.937673e+07,3.741027e+06,-1.937675e+07,1.000000,0.000000,0.000000
25%,53930.000000,54658.000000,3.755733e+06,-1.927453e+07,3.755173e+06,-1.927946e+07,312.000000,1413.419774,3.763758
50%,54605.000000,55398.000000,3.760945e+06,-1.922949e+07,3.760848e+06,-1.922872e+07,596.000000,10453.900774,18.174478
75%,55564.000000,56351.000000,3.768055e+06,-1.917067e+07,3.768347e+06,-1.916246e+07,1120.000000,40495.060729,52.525586
max,57572.000000,57600.000000,3.777099e+06,-1.904709e+07,3.777048e+06,-1.904668e+07,19140.000000,320346.373937,499.325821


In [6]:
## Finds if the hash is in the good devices found in df4
df2['Good_hash']=df2['hash'].isin(Good_Hashs['hash'])

print(df2.head(10))

In [7]:
df3=df2[df2['Good_hash']]
print(df3.head())

                                 hash  \
0  0000a8602cf2def930488dee7cdad104_1   
1  0000a8602cf2def930488dee7cdad104_1   
2  0000a8602cf2def930488dee7cdad104_1   
3  0000a8602cf2def930488dee7cdad104_1   
4  0000a8602cf2def930488dee7cdad104_1   

                               trajectory_id  time_entry  time_exit  \
0  traj_0000a8602cf2def930488dee7cdad104_1_0     25471.0    25712.0   
1  traj_0000a8602cf2def930488dee7cdad104_1_1     26434.0    26742.0   
2  traj_0000a8602cf2def930488dee7cdad104_1_2     28412.0    29005.0   
3  traj_0000a8602cf2def930488dee7cdad104_1_3     29870.0    31043.0   
4  traj_0000a8602cf2def930488dee7cdad104_1_4     52689.0    52689.0   

        x_entry       y_entry        x_exit        y_exit  Good_hash  
0  3.751014e+06 -1.909398e+07  3.750326e+06 -1.913634e+07       True  
1  3.743937e+06 -1.932247e+07  3.744975e+06 -1.931966e+07       True  
2  3.744868e+06 -1.929356e+07  3.744816e+06 -1.929284e+07       True  
3  3.744880e+06 -1.929229e+07  3.744809e+0

In [8]:
df3['deltaT']=df3['time_exit']-df3['time_entry']
df3['real_trip']=df3['deltaT']>0

print(df3.head())

df3.describe()

                                 hash  \
0  0000a8602cf2def930488dee7cdad104_1   
1  0000a8602cf2def930488dee7cdad104_1   
2  0000a8602cf2def930488dee7cdad104_1   
3  0000a8602cf2def930488dee7cdad104_1   
4  0000a8602cf2def930488dee7cdad104_1   

                               trajectory_id  time_entry  time_exit  \
0  traj_0000a8602cf2def930488dee7cdad104_1_0     25471.0    25712.0   
1  traj_0000a8602cf2def930488dee7cdad104_1_1     26434.0    26742.0   
2  traj_0000a8602cf2def930488dee7cdad104_1_2     28412.0    29005.0   
3  traj_0000a8602cf2def930488dee7cdad104_1_3     29870.0    31043.0   
4  traj_0000a8602cf2def930488dee7cdad104_1_4     52689.0    52689.0   

        x_entry       y_entry        x_exit        y_exit  Good_hash  deltaT  \
0  3.751014e+06 -1.909398e+07  3.750326e+06 -1.913634e+07       True   241.0   
1  3.743937e+06 -1.932247e+07  3.744975e+06 -1.931966e+07       True   308.0   
2  3.744868e+06 -1.929356e+07  3.744816e+06 -1.929284e+07       True   593.0   
3  3.7

,time_entry,time_exit,x_entry,y_entry,x_exit,y_exit,deltaT
count,420991.000000,420991.000000,4.209910e+05,4.209910e+05,4.209910e+05,4.209910e+05,420991.000000
mean,39370.446411,39864.308280,3.760712e+06,-1.922107e+07,3.760736e+06,-1.922130e+07,493.861869
std,13708.143826,13801.418062,9.086057e+03,7.620081e+04,9.016128e+03,7.504009e+04,717.253979
min,0.000000,0.000000,3.741031e+06,-1.938291e+07,3.740998e+06,-1.937688e+07,0.000000
25%,30799.000000,31344.000000,3.755528e+06,-1.927315e+07,3.755699e+06,-1.927165e+07,0.000000
50%,42306.000000,42746.000000,3.760551e+06,-1.922974e+07,3.760556e+06,-1.922975e+07,279.000000
75%,50936.000000,51397.000000,3.767816e+06,-1.916928e+07,3.767775e+06,-1.917126e+07,701.000000
max,57572.000000,57600.000000,3.777099e+06,-1.904266e+07,3.777055e+06,-1.904668e+07,24577.000000


In [9]:
#Calculate velocity and remove >500 and under 1000000

df3['delta_X']=np.sqrt((df3['x_exit']-df3['x_entry'])**2+(df3['y_exit']-df3['y_entry'])**2)
df3['vel']=df3['delta_X']/df3['deltaT']
df3['not_reasonable_vel']=(df3['vel']>max_vel) 
df4=df3[~df3['not_reasonable_vel']]
print(df4.head())
df4.describe()

#df4 is all trips which have a valid last trip (is in df4 in the preprocess data file), 
#don't have velocity over 500 but may have delta time = 0

                                 hash  \
0  0000a8602cf2def930488dee7cdad104_1   
1  0000a8602cf2def930488dee7cdad104_1   
2  0000a8602cf2def930488dee7cdad104_1   
3  0000a8602cf2def930488dee7cdad104_1   
4  0000a8602cf2def930488dee7cdad104_1   

                               trajectory_id  time_entry  time_exit  \
0  traj_0000a8602cf2def930488dee7cdad104_1_0     25471.0    25712.0   
1  traj_0000a8602cf2def930488dee7cdad104_1_1     26434.0    26742.0   
2  traj_0000a8602cf2def930488dee7cdad104_1_2     28412.0    29005.0   
3  traj_0000a8602cf2def930488dee7cdad104_1_3     29870.0    31043.0   
4  traj_0000a8602cf2def930488dee7cdad104_1_4     52689.0    52689.0   

        x_entry       y_entry        x_exit        y_exit  Good_hash  deltaT  \
0  3.751014e+06 -1.909398e+07  3.750326e+06 -1.913634e+07       True   241.0   
1  3.743937e+06 -1.932247e+07  3.744975e+06 -1.931966e+07       True   308.0   
2  3.744868e+06 -1.929356e+07  3.744816e+06 -1.929284e+07       True   593.0   
3  3.7

,time_entry,time_exit,x_entry,y_entry,x_exit,y_exit,deltaT,delta_X,vel
count,419708.000000,419708.000000,4.197080e+05,4.197080e+05,4.197080e+05,4.197080e+05,419708.000000,419708.000000,266588.000000
mean,39374.692632,39869.929439,3.760711e+06,-1.922106e+07,3.760736e+06,-1.922130e+07,495.236807,18547.807075,40.367023
std,13707.940146,13801.339324,9.085740e+03,7.618465e+04,9.015452e+03,7.501930e+04,717.906111,38890.314956,58.206452
min,0.000000,0.000000,3.741031e+06,-1.938291e+07,3.740998e+06,-1.937688e+07,0.000000,0.000000,0.000000
25%,30799.000000,31345.750000,3.755529e+06,-1.927306e+07,3.755701e+06,-1.927159e+07,0.000000,0.000000,3.672657
50%,42310.000000,42752.000000,3.760551e+06,-1.922974e+07,3.760556e+06,-1.922975e+07,282.000000,948.639317,17.964438
75%,50942.000000,51407.000000,3.767815e+06,-1.916928e+07,3.767774e+06,-1.917129e+07,703.000000,16935.683059,53.125464
max,57572.000000,57600.000000,3.777099e+06,-1.904266e+07,3.777055e+06,-1.904668e+07,24577.000000,324166.394912,499.325821


In [10]:

# save the normalisation variables
Norm_Values= {'X_min':  np.minimum( df4['x_exit'].min() , df4['x_entry'].min() ),
        'X_max':  np.maximum(df4['x_exit'].max(),df4['x_entry'].max()),
        'Y_min':  np.minimum(df4['y_exit'].min(),df4['y_entry'].min()),
        'Y_max':  np.maximum(df4['y_exit'].max(),df4['y_entry'].max()),
        'T_min':  df4['time_entry'].min(),
        'T_max':  16*60*60   
        }
print(Norm_Values)

#save in pickle
norm_out = open("normvalues.pickle","wb")
pickle.dump(Norm_Values,norm_out)
norm_out.close()

{'X_min': 3740998.35481912, 'X_max': 3777099.2656833767, 'Y_min': -19382914.9809002, 'Y_max': -19042656.658487003, 'T_min': 0.0, 'T_max': 57600}


In [11]:
#Normalize data

df4['Xentry_norm'] = (df4['x_entry']-Norm_Values['X_min'])/(Norm_Values['X_max'] - Norm_Values['X_min'])
df4['Xexit_norm'] = (df4['x_exit']-Norm_Values['X_min'])/(Norm_Values['X_max'] - Norm_Values['X_min'])

df4['Yentry_norm'] = (df4['y_entry']-Norm_Values['Y_min'])/(Norm_Values['Y_max'] - Norm_Values['Y_min'])
df4['Yexit_norm'] = (df4['y_exit']-Norm_Values['Y_min'])/(Norm_Values['Y_max'] - Norm_Values['Y_min'])

df4['T_entry_norm'] = (df4['time_entry'] - Norm_Values['T_min'])/ (Norm_Values['T_max'] - Norm_Values['T_min'])
df4['T_exit_norm'] = (df4['time_exit'] - Norm_Values['T_min'])/ (Norm_Values['T_max'] - Norm_Values['T_min'])


print(df4.head())

                                 hash  \
0  0000a8602cf2def930488dee7cdad104_1   
1  0000a8602cf2def930488dee7cdad104_1   
2  0000a8602cf2def930488dee7cdad104_1   
3  0000a8602cf2def930488dee7cdad104_1   
4  0000a8602cf2def930488dee7cdad104_1   

                               trajectory_id  time_entry  time_exit  \
0  traj_0000a8602cf2def930488dee7cdad104_1_0     25471.0    25712.0   
1  traj_0000a8602cf2def930488dee7cdad104_1_1     26434.0    26742.0   
2  traj_0000a8602cf2def930488dee7cdad104_1_2     28412.0    29005.0   
3  traj_0000a8602cf2def930488dee7cdad104_1_3     29870.0    31043.0   
4  traj_0000a8602cf2def930488dee7cdad104_1_4     52689.0    52689.0   

        x_entry       y_entry        x_exit        y_exit  Good_hash  deltaT  \
0  3.751014e+06 -1.909398e+07  3.750326e+06 -1.913634e+07       True   241.0   
1  3.743937e+06 -1.932247e+07  3.744975e+06 -1.931966e+07       True   308.0   
2  3.744868e+06 -1.929356e+07  3.744816e+06 -1.929284e+07       True   593.0   
3  3.7

dftemp = df4[df4['real_trip']]
dftemp.describe()

temp=df4.groupby('hash')[['trajectory_id']].count()

temp2=temp.groupby('trajectory_id').size()

In [12]:
#Create a copy which will be for first trips only

df4_first = df4.copy()
#df4_first.describe()

In [13]:
#Drop duplicates keeping only the first trip for each hash

df4_first = df4_first.drop_duplicates(subset='hash', keep='first')
df4_first.head()

,hash,trajectory_id,time_entry,time_exit,x_entry,y_entry,x_exit,y_exit,Good_hash,deltaT,real_trip,delta_X,vel,not_reasonable_vel,Xentry_norm,Xexit_norm,Yentry_norm,Yexit_norm,T_entry_norm,T_exit_norm
0,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_0,25471.0,25712.0,3.751014e+06,-1.909398e+07,3.750326e+06,-1.913634e+07,True,241.0,True,42364.863118,175.787814,False,0.277428,0.258372,0.849162,0.724670,0.442205,0.446389
6,0000cf177130469eeac79f67b6bcf3df_9,traj_0000cf177130469eeac79f67b6bcf3df_9_0,52163.0,52163.0,3.749450e+06,-1.926506e+07,3.749450e+06,-1.926506e+07,True,0.0,False,0.000000,NaN,False,0.234104,0.234104,0.346383,0.346383,0.905608,0.905608
10,0001f97b99a80f18f62e2d44e54ef33d_3,traj_0001f97b99a80f18f62e2d44e54ef33d_3_0,41826.0,42728.0,3.771461e+06,-1.910413e+07,3.757004e+06,-1.929698e+07,True,902.0,True,193384.920615,214.395699,False,0.843817,0.443355,0.819321,0.252564,0.726146,0.741806
29,0002974025ed311b78ad49904a31d4ea_25,traj_0002974025ed311b78ad49904a31d4ea_25_0,49925.0,50227.0,3.744675e+06,-1.931587e+07,3.746521e+06,-1.930575e+07,True,302.0,True,10281.598238,34.045027,False,0.101850,0.152976,0.197043,0.226769,0.866753,0.871997
42,00037fa9d812de1961f1e2603ad4b22b_25,traj_00037fa9d812de1961f1e2603ad4b22b_25_0,45667.0,45667.0,3.773369e+06,-1.911817e+07,3.773369e+06,-1.911817e+07,True,0.0,False,0.000000,NaN,False,0.896670,0.896670,0.778068,0.778068,0.792830,0.792830


In [14]:
#Drop duplicates keeping only the last trip for each hash

df4_last = df4.copy()
df4_last = df4_last.drop_duplicates(subset='hash', keep='last')
df4_last.head()

,hash,trajectory_id,time_entry,time_exit,x_entry,y_entry,x_exit,y_exit,Good_hash,deltaT,real_trip,delta_X,vel,not_reasonable_vel,Xentry_norm,Xexit_norm,Yentry_norm,Yexit_norm,T_entry_norm,T_exit_norm
5,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_5,54151.0,55113.0,3.744945e+06,-1.928183e+07,3.744785e+06,-1.928148e+07,True,962.0,True,389.513093,0.404899,False,0.109312,0.104903,0.297072,0.298117,0.940122,0.956823
9,0000cf177130469eeac79f67b6bcf3df_9,traj_0000cf177130469eeac79f67b6bcf3df_9_3,54032.0,55788.0,3.749088e+06,-1.926605e+07,3.749610e+06,-1.926594e+07,True,1756.0,True,534.474570,0.304370,False,0.224083,0.238545,0.343459,0.343795,0.938056,0.968542
11,0001f97b99a80f18f62e2d44e54ef33d_3,traj_0001f97b99a80f18f62e2d44e54ef33d_3_1,52475.0,55191.0,3.758738e+06,-1.937594e+07,3.769687e+06,-1.914258e+07,True,2716.0,True,233620.804947,86.016497,False,0.491397,0.794665,0.020498,0.706342,0.911024,0.958177
31,0002974025ed311b78ad49904a31d4ea_25,traj_0002974025ed311b78ad49904a31d4ea_25_3,53889.0,54310.0,3.745723e+06,-1.930532e+07,3.746480e+06,-1.935579e+07,True,421.0,True,50480.282773,119.905660,False,0.130866,0.151835,0.228055,0.079713,0.935573,0.942882
45,00037fa9d812de1961f1e2603ad4b22b_25,traj_00037fa9d812de1961f1e2603ad4b22b_25_4,54829.0,55535.0,3.768952e+06,-1.914929e+07,3.768413e+06,-1.915680e+07,True,706.0,True,7527.557492,10.662263,False,0.774314,0.759389,0.686610,0.664544,0.951892,0.964149


In [15]:
#Identify duplicates except for the last one

df4['has_dup']= df4['hash'].duplicated(keep=False)
df4['is_last'] = df4['trajectory_id'].isin(df4_last['trajectory_id'])

#df4.head(10)

#Remove the last duplicate (only if there ARE duplicates)
df4_2last = df4.copy()
df4_2last = df4_2last[~((df4_2last['has_dup'])&(df4_2last['is_last']))]
#df4_2last.head(10)

#Remove duplicates keeping only the last one (really the second to last)
df4_2last = df4_2last.drop_duplicates(subset='hash', keep='last')
df4_2last.head(10)

,hash,trajectory_id,time_entry,time_exit,x_entry,y_entry,x_exit,y_exit,Good_hash,deltaT,...,vel,not_reasonable_vel,Xentry_norm,Xexit_norm,Yentry_norm,Yexit_norm,T_entry_norm,T_exit_norm,has_dup,is_last
4,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_4,52689.0,52689.0,3.744909e+06,-1.928558e+07,3.744909e+06,-1.928558e+07,True,0.0,...,NaN,False,0.108326,0.108326,0.286047,0.286047,0.914740,0.914740,True,False
8,0000cf177130469eeac79f67b6bcf3df_9,traj_0000cf177130469eeac79f67b6bcf3df_9_2,53414.0,53414.0,3.749042e+06,-1.926632e+07,3.749042e+06,-1.926632e+07,True,0.0,...,NaN,False,0.222818,0.222818,0.342652,0.342652,0.927326,0.927326,True,False
10,0001f97b99a80f18f62e2d44e54ef33d_3,traj_0001f97b99a80f18f62e2d44e54ef33d_3_0,41826.0,42728.0,3.771461e+06,-1.910413e+07,3.757004e+06,-1.929698e+07,True,902.0,...,214.395699,False,0.843817,0.443355,0.819321,0.252564,0.726146,0.741806,True,False
30,0002974025ed311b78ad49904a31d4ea_25,traj_0002974025ed311b78ad49904a31d4ea_25_1,50940.0,51249.0,3.747023e+06,-1.930239e+07,3.748910e+06,-1.929660e+07,True,309.0,...,19.725742,False,0.166883,0.219149,0.236645,0.253679,0.884375,0.889740,True,False
44,00037fa9d812de1961f1e2603ad4b22b_25,traj_00037fa9d812de1961f1e2603ad4b22b_25_2,49831.0,50358.0,3.768547e+06,-1.915444e+07,3.768412e+06,-1.915670e+07,True,527.0,...,4.296241,False,0.763090,0.759359,0.671484,0.664842,0.865122,0.874271,True,False
48,0004e4995492d96bcf550a884e925ecc_25,traj_0004e4995492d96bcf550a884e925ecc_25_2,39212.0,39828.0,3.774350e+06,-1.919904e+07,3.775669e+06,-1.919093e+07,True,616.0,...,13.341680,False,0.923836,0.960376,0.540386,0.564227,0.680764,0.691458,True,False
66,00068f505957087127de6d86106d7ae0_31,traj_00068f505957087127de6d86106d7ae0_31_8,52172.0,52172.0,3.766789e+06,-1.922233e+07,3.766789e+06,-1.922233e+07,True,0.0,...,NaN,False,0.714400,0.714400,0.471956,0.471956,0.905764,0.905764,True,False
75,0008cd399c4252b370fdde881e616fe2_29,traj_0008cd399c4252b370fdde881e616fe2_29_7,52181.0,52181.0,3.761034e+06,-1.930394e+07,3.761034e+06,-1.930394e+07,True,0.0,...,NaN,False,0.554980,0.554980,0.232113,0.232113,0.905920,0.905920,True,False
78,0008d48d85e5c1fcc1d99f90b0de3193_23,traj_0008d48d85e5c1fcc1d99f90b0de3193_23_1,47447.0,51029.0,3.761024e+06,-1.922753e+07,3.744969e+06,-1.929782e+07,True,3582.0,...,20.126641,False,0.554702,0.109991,0.456656,0.250097,0.823733,0.885920,True,False
100,000a77e0fb8e6cc8915df6d5fa35d7c9_31,traj_000a77e0fb8e6cc8915df6d5fa35d7c9_31_7,48110.0,48110.0,3.746588e+06,-1.936301e+07,3.746588e+06,-1.936301e+07,True,0.0,...,NaN,False,0.154826,0.154826,0.058493,0.058493,0.835243,0.835243,True,False


In [16]:
print(df4_first.shape)
print(df4_2last.shape)
print(df4_last.shape)


(66762, 20)
(66762, 22)
(66762, 20)


In [22]:
#renaming columns

df4_first.rename(index=str, columns={'trajectory_id': 'traj_1', 
                                  'Xentry_norm': 'Xi1','Xexit_norm': 'Xf1', 
                                  'Yentry_norm': 'Yi1','Yexit_norm':'Yf1',
                                  'T_entry_norm':'Ti1', 'T_exit_norm':'Tf1'},inplace= True)

#df4_first.head()

df4_2last.rename(index=str, columns={'trajectory_id': 'traj_2', 
                                  'Xentry_norm': 'Xi2','Xexit_norm': 'Xf2', 
                                  'Yentry_norm': 'Yi2','Yexit_norm':'Yf2',
                                  'T_entry_norm':'Ti2', 'T_exit_norm':'Tf2'},inplace= True)

#df4_2last.head()

df4_last.rename(index=str, columns={'trajectory_id': 'traj_3', 
                                  'Xentry_norm': 'Xi3','Xexit_norm': 'Xf3', 
                                  'Yentry_norm': 'Yi3','Yexit_norm':'Yf3',
                                  'T_entry_norm':'Ti3', 'T_exit_norm':'Tf3'},inplace= True)

#df4_last.head()

,hash,traj_3,time_entry,time_exit,x_entry,y_entry,x_exit,y_exit,Good_hash,deltaT,real_trip,delta_X,vel,not_reasonable_vel,Xi3,Xf3,Yi3,Yf3,Ti3,Tf3
5,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_5,54151.0,55113.0,3.744945e+06,-1.928183e+07,3.744785e+06,-1.928148e+07,True,962.0,True,389.513093,0.404899,False,0.109312,0.104903,0.297072,0.298117,0.940122,0.956823
9,0000cf177130469eeac79f67b6bcf3df_9,traj_0000cf177130469eeac79f67b6bcf3df_9_3,54032.0,55788.0,3.749088e+06,-1.926605e+07,3.749610e+06,-1.926594e+07,True,1756.0,True,534.474570,0.304370,False,0.224083,0.238545,0.343459,0.343795,0.938056,0.968542
11,0001f97b99a80f18f62e2d44e54ef33d_3,traj_0001f97b99a80f18f62e2d44e54ef33d_3_1,52475.0,55191.0,3.758738e+06,-1.937594e+07,3.769687e+06,-1.914258e+07,True,2716.0,True,233620.804947,86.016497,False,0.491397,0.794665,0.020498,0.706342,0.911024,0.958177
31,0002974025ed311b78ad49904a31d4ea_25,traj_0002974025ed311b78ad49904a31d4ea_25_3,53889.0,54310.0,3.745723e+06,-1.930532e+07,3.746480e+06,-1.935579e+07,True,421.0,True,50480.282773,119.905660,False,0.130866,0.151835,0.228055,0.079713,0.935573,0.942882
45,00037fa9d812de1961f1e2603ad4b22b_25,traj_00037fa9d812de1961f1e2603ad4b22b_25_4,54829.0,55535.0,3.768952e+06,-1.914929e+07,3.768413e+06,-1.915680e+07,True,706.0,True,7527.557492,10.662263,False,0.774314,0.759389,0.686610,0.664544,0.951892,0.964149


In [30]:
#Merge df first and second to last

df5 = pd.merge(df4_first,df4_2last,on='hash')


#merge dfs with last
df6 = pd.merge(df5,df4_last,on='hash')

df7 = pd.merge(df6,Good_Hashs[['hash','incity']],on = 'hash')
print(df7.head())
df7.shape

                                  hash  \
0   0000a8602cf2def930488dee7cdad104_1   
1   0000cf177130469eeac79f67b6bcf3df_9   
2   0001f97b99a80f18f62e2d44e54ef33d_3   
3  0002974025ed311b78ad49904a31d4ea_25   
4  00037fa9d812de1961f1e2603ad4b22b_25   

                                       traj_1  time_entry_x  time_exit_x  \
0   traj_0000a8602cf2def930488dee7cdad104_1_0       25471.0      25712.0   
1   traj_0000cf177130469eeac79f67b6bcf3df_9_0       52163.0      52163.0   
2   traj_0001f97b99a80f18f62e2d44e54ef33d_3_0       41826.0      42728.0   
3  traj_0002974025ed311b78ad49904a31d4ea_25_0       49925.0      50227.0   
4  traj_00037fa9d812de1961f1e2603ad4b22b_25_0       45667.0      45667.0   

      x_entry_x     y_entry_x      x_exit_x      y_exit_x  Good_hash_x  \
0  3.751014e+06 -1.909398e+07  3.750326e+06 -1.913634e+07         True   
1  3.749450e+06 -1.926506e+07  3.749450e+06 -1.926506e+07         True   
2  3.771461e+06 -1.910413e+07  3.757004e+06 -1.929698e+07         Tr

(66762, 61)

In [35]:
#convert to numpy arrays

#reate a np-array with entry data AND target

entry_target_columns = ['Xi1','Xf1','Yi1','Yf1','Ti1','Tf1',
             'Xi2','Xf2','Yi2','Yf2','Ti2','Tf2',
             'Xi3','Yi3','Ti3','Tf3','incity']
np_all = df7[entry_target_columns].values

print('Before:',np_all[0:5])

Before:\ [[0.2774283071230577 0.2583718778908547 0.8491617041275193
  0.7246701805339721 0.4422048611111111 0.4463888888888889
  0.10832562442184815 0.10832562442184815 0.28604732835719404
  0.28604732835719404 0.9147395833333334 0.9147395833333334
  0.1093123650940354 0.29707174472244896 0.9401215277777778
  0.9568229166666666 False]
 [0.23410422448347898 0.23410422448347898 0.3463834153331232
  0.3463834153331232 0.9056076388888888 0.9056076388888888
  0.22281837804503427 0.22281837804503427 0.3426516898838349
  0.3426516898838349 0.9273263888888889 0.9273263888888889
  0.2240826395312645 0.34345861209303363 0.9380555555555555
  0.9685416666666666 False]
 [0.843817452975742 0.44335491828556456 0.819320652588683
  0.2525636902382412 0.7261458333333334 0.7418055555555556
  0.843817452975742 0.44335491828556456 0.819320652588683
  0.2525636902382412 0.7261458333333334 0.7418055555555556
  0.49139685476414496 0.020498492084227 0.9110243055555556
  0.9581770833333333 False]
 [0.1018501387

In [37]:
total_entries = df7.shape[0]
print('Total entries:',total_entries)
tot_in_city = df7.incity.sum()
print('Total entries in the city:',tot_in_city)
tot_non_city = total_entries - tot_in_city
print('Total entries not in the city:',tot_non_city)

Total entries: 66762
Total entries in the city: 16804
Total entries not in the city: 49958


In [42]:
#Create empty np arrays City2 and NonCity2 and fill them with the corresponding rows from np_all

City = np.zeros((tot_in_city,17))
NonCity = np.zeros((tot_non_city,17))

num_non_city = 0
num_in_city = 0

for index in range(total_entries):  # iterate over the sequential data
        if np_all[index,16] == False:  # if it's not in the city
            NonCity[num_non_city]=np_all[index] #add entries to the NonCity np array
            num_non_city +=1
        elif np_all[index,16] == True:  # otherwise if the journey ends in the city
            City[num_in_city]=np_all[index] #add entries to the City np array
            num_in_city +=1
            
print('Total entries:',total_entries)
print('Number in city:', num_in_city)
print('Number of non city:', num_non_city)


Total entries: 66762
Number in city: 16804
Number of non city: 49958


In [43]:
#Shuffle both City and NonCity arrays
City = shuffle(City)
NonCity = shuffle(NonCity)

lower = min(tot_in_city, tot_non_city) #Are there more in or out of the city

City = City[:lower,:] # make sure both lists are only up to the shortest length.
NonCity = NonCity[:lower,:] # make sure both lists are only up to the shortest length.

#print(City.shape)
#print(NonCity.shape)

np_all_balanced = np.concatenate((City,NonCity)) #Stick them together
#print(np_all_balanced[16802:16806,:])

np_all_balanced = shuffle(np_all_balanced)
#print(np_all_balanced[16802:16806,:])

In [45]:
#Separate the np array with 50% in the city with entry and target data into 4 arrays: train and test, entry and target
test_perc = 0.2

size_total = np_all_balanced.shape[0]
num_train = (1-test_perc)*size_total
print('Total size:',size_total,' Number for training: ',int(num_train))


np_xtrain = np_all_balanced[0:int(num_train),0:16]
np_xtest = np_all_balanced[int(num_train):,0:16]

np_ytrain = np_all_balanced[0:int(num_train),16]
np_ytest = np_all_balanced[int(num_train):,16]

print(np_xtrain.shape, np_xtest.shape, np_ytrain.shape, np_ytest.shape)

Total size: 33608  Number for training:  26886
(26886, 16) (6722, 16) (26886,) (6722,)


In [46]:
#save in pickles
pickle_out = open("Xtrain.pickle", "wb")
pickle.dump(np_xtrain,pickle_out)
pickle_out.close()

pickle_out = open("Ytrain.pickle", "wb")
pickle.dump(np_ytrain,pickle_out)
pickle_out.close()

pickle_out = open("Xtest.pickle", "wb")
pickle.dump(np_xtest,pickle_out)
pickle_out.close()

pickle_out = open("Ytest.pickle", "wb")
pickle.dump(np_ytest,pickle_out)
pickle_out.close()